In [ ]:
import numpy as np
import cv2
from scipy.spatial import KDTree
import os
import pandas as pd
import matplotlib.pyplot as plt

import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr, spearmanr
import numpy as np
import json

# evaluation on inpainted dataset

In [ ]:
def evaluate_width_directly(DATASET_NAME, EXPERIMENT_NAME):
    segmentation_folder_path = os.path.join("data", DATASET_NAME, EXPERIMENT_NAME, "segmentations")
    print(segmentation_folder_path)
    localized_folder_path = os.path.join("data", DATASET_NAME, EXPERIMENT_NAME, "localized")
    df = pd.read_csv(os.path.join("data", "forecast_key_dataset.csv"))
    plaque_width_results = []
    
    for img_name in os.listdir(segmentation_folder_path):
        print(img_name)
    # segmentation_path = 'path_to_your_segmentation.png'  # Replace with your segmentation file path
        if img_name in df['img_name'].values:
            # Filter the DataFrame to get the corresponding row
            row = df[df['img_name'] == img_name]
            plaque_width_value = row['plaque_width'].values[0]
            try:
                plaque_width_value = pd.to_numeric(plaque_width_value)
            except ValueError:
                continue
        else:
            continue

        segmentation_path = os.path.join(segmentation_folder_path, img_name)
        localized_path = os.path.join(localized_folder_path, img_name)
        # print(localized_path)
        segmentation = cv2.imread(segmentation_path)
        localization = cv2.imread(localized_path)
        # print(localization.shape)
        new_height, new_width, channels = localization.shape
        # Convert the segmentation to RGB from BGR (OpenCV loads segmentations in BGR format)
        segmentation = cv2.cvtColor(segmentation, cv2.COLOR_BGR2RGB)

        # Display the image in a window
        # plt.imshow(segmentation)
        # plt.axis('off')  # Turn off axis labels and ticks
        # plt.show()

        segmentation = cv2.resize(segmentation, (new_width, new_height))
        # Define the color threshold for green and blue in RGB
        lower_green = np.array([0, 100, 0], dtype="uint8")
        upper_green = np.array([100, 255, 100], dtype="uint8")
        lower_blue = np.array([0, 0, 100], dtype="uint8")
        upper_blue = np.array([100, 100, 255], dtype="uint8")

        # Create masks for green and blue colors
        mask_green = cv2.inRange(segmentation, lower_green, upper_green)
        mask_blue = cv2.inRange(segmentation, lower_blue, upper_blue)

        # Find all green pixels
        green_pixels = np.column_stack(np.where(mask_green > 0))

        # Find all blue pixels
        blue_pixels = np.column_stack(np.where(mask_blue > 0))

        # Compute the distances from all green points to all blue points and find the longest shortest distance
        longest_shortest_distance = 0
        longest_pair = None

        # Create a KDTree with blue pixels for efficient distance computation
        tree = KDTree(blue_pixels)

        # Loop over all green pixels to find the closest blue pixel
        for green_pixel in green_pixels:
            distance, index = tree.query(green_pixel)
            if distance > longest_shortest_distance:
                longest_shortest_distance = distance
                longest_pair = (green_pixel, blue_pixels[index])

        # Draw a white line between the furthest green pixel and the closest blue pixel
        if longest_pair is not None:
            cv2.line(segmentation, tuple(longest_pair[0][::-1]), tuple(longest_pair[1][::-1]), (255, 255, 255), 2)

        # Save the resulting segmentation
        os.makedirs(os.path.join("data", "measured", EXPERIMENT_NAME, "images"), exist_ok= True)
        output_path = os.path.join("data", "measured", EXPERIMENT_NAME,"images", img_name)  # Replace with your desired output file path

        # Display the image in a window
        # plt.imshow(segmentation)
        # plt.axis('off')  # Turn off axis labels and ticks
        # plt.show()

        segmentation = cv2.cvtColor(segmentation, cv2.COLOR_RGB2BGR)
        cv2.imwrite(output_path, segmentation)


        
        # print(f"The longest shortest distance is: {longest_shortest_distance} pixels")
        # print(f'distance in mm: {longest_shortest_distance/17.2}')
        # print(f'label in mmm: {df[df["img_name"] == img_name]["plaque_width"].iloc[0]}')
        
        # BECAUSE THE SEGMENTATION IS 256x256 and image is 380x380
        # longest_shortest_distance = longest_shortest_distance * 380/256

        plaque_width_results.append((img_name, df[df["img_name"] == img_name]["plaque_width"].iloc[0], longest_shortest_distance/17.2, longest_shortest_distance))

    df = pd.DataFrame(plaque_width_results, columns=['name', 'true_width', 'predicted_width_mm', 'predicted_width_pixels'])
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)
    df.to_csv(os.path.join("data", "measured", EXPERIMENT_NAME, "results.csv"))

    mae = mean_absolute_error(df['true_width'], df['predicted_width_mm'])
    mse = mean_squared_error(df['true_width'], df['predicted_width_mm'])
    rmse = np.sqrt(mse)
    r2 = r2_score(df['true_width'], df['predicted_width_mm'])
    pearson_corr, _ = pearsonr(df['true_width'], df['predicted_width_mm'])
    spearman_corr, _ = spearmanr(df['true_width'], df['predicted_width_mm'])

    # Print Results
    print(f"Mean Absolute Error (MAE): {mae}")
    print(f"Mean Squared Error (MSE): {mse}")
    print(f"Root Mean Squared Error (RMSE): {rmse}")
    print(f"R-squared: {r2}")
    print(f"Pearson Correlation Coefficient: {pearson_corr}")
    print(f"Spearman's Rank Correlation Coefficient: {spearman_corr}")

    result_statistics = {
        "mae": mae,
        "mse": mse,
        "rmse": rmse,
        "r2": r2,
        "pearson_corr": pearson_corr,
        "spearman_corr": spearman_corr
    }

    with open(os.path.join("data", "measured", EXPERIMENT_NAME, "statistics.json"), "w") as f:
        json.dump(result_statistics, f)
        

In [ ]:
DATASET_NAME = "carotid_key_inpainted_dataset"
EXPERIMENT_NAME = "ACNN_FIX"
evaluate_width_directly(DATASET_NAME, EXPERIMENT_NAME)
# EXPERIMENT_NAME = "MAAG_FIX"
# evaluate_width_directly(DATASET_NAME, EXPERIMENT_NAME)
# EXPERIMENT_NAME = "SEGAN_FIX"
# evaluate_width_directly(DATASET_NAME, EXPERIMENT_NAME)
# EXPERIMENT_NAME = "UNETPAPER_FIX"
# evaluate_width_directly(DATASET_NAME, EXPERIMENT_NAME)
# EXPERIMENT_NAME = "ACNN_DYNAMIC"
# evaluate_width_directly(DATASET_NAME, EXPERIMENT_NAME)
# EXPERIMENT_NAME = "MAAG_DYNAMIC"
# evaluate_width_directly(DATASET_NAME, EXPERIMENT_NAME)
# EXPERIMENT_NAME = "SEGAN_DYNAMIC"
# evaluate_width_directly(DATASET_NAME, EXPERIMENT_NAME)
# EXPERIMENT_NAME = "UNETPAPER_DYNAMIC"
# evaluate_width_directly(DATASET_NAME, EXPERIMENT_NAME)

# evaluation on anotated dataset DOCTORS ANOTATION

In [ ]:
DATASET_NAME = "carotid_key_inpainted_dataset"
EXPERIMENT_NAME = "doctors_segmentation"
segmentation_folder_path = os.path.join("data", "segmentation_data", "references", "trans")
localized_folder_path = os.path.join("data", "segmentation_data", "data", "trans")
df = pd.read_csv(os.path.join("data", "data_with_attributes.csv"))
plaque_width_results = []

for img_name in os.listdir(segmentation_folder_path):
# segmentation_path = 'path_to_your_segmentation.png'  # Replace with your segmentation file path
    if img_name in df['img_name'].values:
        # Filter the DataFrame to get the corresponding row
        row = df[df['img_name'] == img_name]
        plaque_width_value = row['plaque_width'].values[0]
        try:
            plaque_width_numeric = pd.to_numeric(plaque_width_value)
        except ValueError:
            continue
    else:
        continue

    # print(df[df['img_name'] == img_name]['plaque_width'].values[0])
    segmentation_path = os.path.join(segmentation_folder_path, img_name)
    localized_path = os.path.join(localized_folder_path, img_name)
    # print(localized_path)
    segmentation = cv2.imread(segmentation_path)
    localization = cv2.imread(localized_path)
    # print(localization.shape)
    # new_height, new_width, channels = localization.shape
    # Convert the segmentation to RGB from BGR (OpenCV loads segmentations in BGR format)
    segmentation = cv2.cvtColor(segmentation, cv2.COLOR_BGR2RGB)

    # Display the image in a window
    # plt.imshow(segmentation)
    # plt.axis('off')  # Turn off axis labels and ticks
    # plt.show()

    # segmentation = cv2.resize(segmentation, (new_width, new_height))
    # Define the color threshold for green and blue in RGB
    lower_green = np.array([0, 100, 0], dtype="uint8")
    upper_green = np.array([100, 255, 100], dtype="uint8")
    lower_blue = np.array([0, 0, 100], dtype="uint8")
    upper_blue = np.array([100, 100, 255], dtype="uint8")

    # Create masks for green and blue colors
    mask_green = cv2.inRange(segmentation, lower_green, upper_green)
    mask_blue = cv2.inRange(segmentation, lower_blue, upper_blue)

    # Find all green pixels
    green_pixels = np.column_stack(np.where(mask_green > 0))

    # Find all blue pixels
    blue_pixels = np.column_stack(np.where(mask_blue > 0))

    # Compute the distances from all green points to all blue points and find the longest shortest distance
    longest_shortest_distance = 0
    longest_pair = None

    # Create a KDTree with blue pixels for efficient distance computation
    tree = KDTree(blue_pixels)

    # Loop over all green pixels to find the closest blue pixel
    for green_pixel in green_pixels:
        distance, index = tree.query(green_pixel)
        if distance > longest_shortest_distance:
            longest_shortest_distance = distance
            longest_pair = (green_pixel, blue_pixels[index])

    # Draw a white line between the furthest green pixel and the closest blue pixel
    if longest_pair is not None:
        cv2.line(segmentation, tuple(longest_pair[0][::-1]), tuple(longest_pair[1][::-1]), (255, 255, 255), 2)

    # Save the resulting segmentation
    os.makedirs(os.path.join("data", "measured", EXPERIMENT_NAME, "images"), exist_ok= True)
    output_path = os.path.join("data", "measured", EXPERIMENT_NAME,"images", img_name)  # Replace with your desired output file path

    # Display the image in a window
    # plt.imshow(segmentation)
    # plt.axis('off')  # Turn off axis labels and ticks
    # plt.show()

    segmentation = cv2.cvtColor(segmentation, cv2.COLOR_RGB2BGR)
    cv2.imwrite(output_path, segmentation)


    
    # print(f"The longest shortest distance is: {longest_shortest_distance} pixels")
    # print(f'distance in mm: {longest_shortest_distance/17.2}')
    # print(f'label in mmm: {df[df["img_name"] == img_name]["plaque_width"].iloc[0]}')
    print(df[df["img_name"] == img_name]["plaque_width"])
    plaque_width_results.append((img_name, float(df[df["img_name"] == img_name]["plaque_width"].iloc[0]), float(longest_shortest_distance/17.2), float(longest_shortest_distance)))

df = pd.DataFrame(plaque_width_results, columns=['name', 'true_width', 'predicted_width_mm', 'predicted_width_pixels'])
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)
df.to_csv(os.path.join("data", "measured", EXPERIMENT_NAME, "results.csv"))

mae = mean_absolute_error(df['true_width'], df['predicted_width_mm'])
mse = mean_squared_error(df['true_width'], df['predicted_width_mm'])
rmse = np.sqrt(mse)
r2 = r2_score(df['true_width'], df['predicted_width_mm'])
pearson_corr, _ = pearsonr(df['true_width'], df['predicted_width_mm'])
spearman_corr, _ = spearmanr(df['true_width'], df['predicted_width_mm'])

# Print Results
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared: {r2}")
print(f"Pearson Correlation Coefficient: {pearson_corr}")
print(f"Spearman's Rank Correlation Coefficient: {spearman_corr}")

result_statistics = {
    "mae": mae,
    "mse": mse,
    "rmse": rmse,
    "r2": r2,
    "pearson_corr": pearson_corr,
    "spearman_corr": spearman_corr
}

with open(os.path.join("data", "measured", EXPERIMENT_NAME, "statistics.json"), "w") as f:
    json.dump(result_statistics, f)
    

In [ ]:
diferences = [abs(item[1] - item[2]) for item in plaque_width_results]
plt.hist(diferences, bins='auto', alpha=0.7, color='blue', edgecolor='black')

# Add titles and labels
plt.title('Histogram of Data')
plt.xlabel('Value')
plt.ylabel('Frequency')

In [ ]:
df = pd.DataFrame(plaque_width_results, columns=['name', 'true_width', 'predicted_width_mm', 'predicted_width_pixels'])
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

# mean = df['true_width'].mean()
# std = df['true_width'].std()

# # Standardize the column
# df['true_width'] = (df['true_width'] - mean) / std

# mean = df['predicted_width_mm'].mean()
# std = df['predicted_width_mm'].std()

# # Standardize the column
# df['predicted_width_mm'] = (df['predicted_width_mm'] - mean) / std



mae = mean_absolute_error(df['true_width'], df['predicted_width_mm'])
mse = mean_squared_error(df['true_width'], df['predicted_width_mm'])
rmse = np.sqrt(mse)
r2 = r2_score(df['true_width'], df['predicted_width_mm'])
pearson_corr, _ = pearsonr(df['true_width'], df['predicted_width_mm'])
spearman_corr, _ = spearmanr(df['true_width'], df['predicted_width_mm'])

# Print Results
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared: {r2}")
print(f"Pearson Correlation Coefficient: {pearson_corr}")
print(f"Spearman's Rank Correlation Coefficient: {spearman_corr}")

diferences = df['true_width'] - df['predicted_width_mm']
plt.hist(diferences, bins=20, alpha=0.7, color='blue', edgecolor='black')

# Add titles and labels
plt.title('Histogram of differences')
plt.xlabel('Value')
plt.ylabel('Frequency')
absolute_diferences = (df['true_width'] - df['predicted_width_mm']).abs()
plt.show()
plt.hist(absolute_diferences, bins=20, alpha=0.7, color='blue', edgecolor='black')

# Add titles and labels
plt.title('Histogram of absolute differences')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.show()


In [ ]:
for i in range(diferences.size):
    print(diferences[i])